##### <p> Samuel Wolfe <br> July 30, 2023 <br> MSBA 206 <br> DMBA Chapter 9a</p>

In [2]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import requests
import io
import matplotlib.pylab as plt
import numpy as np

In [3]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt
from dmba import plotDecisionTree, classificationSummary, regressionSummary

In [4]:
def readFile(url):
    download = requests.get(url).content
    # Reading the downloaded content and turning it into a pandas dataframe
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    return df
def statslist(df):
    dfStats = pd.DataFrame({'Mean' : df.mean(numeric_only=True),
            'SD' : df.std(numeric_only=True),
            'Min' : df.min(),
            'Max' : df.max(),
            'Median' : df.median(numeric_only=True),
            })
    return dfStats
def categorize(df):
    for x in df:
        df[x] = df[x].astype('category')
    return df

In [8]:
dfeBay = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/eBayAuctions.csv?token=GHSAT0AAAAAACFSRBB5ZENCNKWJIE3DVQFIZGGS2IA")

In [9]:
dfeBay

,404: Not Found


In [6]:
dfeBay.Category.value_counts()

AttributeError: 'DataFrame' object has no attribute 'Category'

In [ ]:
dfeBay['Duration'] = dfeBay['Duration'].astype('category')
dfeBay_train, dfeBay_valid = train_test_split(dfeBay,test_size=0.4, random_state=0)

In [ ]:
classTree = DecisionTreeClassifier(criterion="gini", random_state=1, max_depth=7, min_samples_leaf=50)
classTree.fit(dfeBay_train, dfeBay_train)
print("Classes: {}".format(', '.join(classTree.classes_)))
plotDecisionTree(classTree, feature_names=dfeBay.columns[:2], class_names=classTree.classes_)

ValueError: could not convert string to float: 'Toys/Hobbies'